In [1]:
import tensorflow as tf
import keras
from tensorflow import keras
import numpy as np
import kagglehub
import os
import cv2

2024-11-26 11:40:40.323994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/kendallgilbert/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tensorflow.keras import utils
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential

In [3]:
path = kagglehub.dataset_download("grassknoted/asl-alphabet")
# Define train_dir
train_images = os.path.join(path, 'asl_alphabet_train', 'asl_alphabet_train')

# Getting Images
def get_data(data_dir) :
    images = []
    labels = []

    dir_list = os.listdir(data_dir)
    for i in range(len(dir_list)):
        print("Obtaining images of", dir_list[i], "...")
        for image in os.listdir(data_dir + "/" + dir_list[i]):
            img = cv2.imread(data_dir + '/' + dir_list[i] + '/' + image)
            img = cv2.resize(img, (32, 32))
            images.append(img)
            labels.append(i)

    return images, labels

X, y = get_data(train_images)

Obtaining images of R ...
Obtaining images of U ...
Obtaining images of I ...
Obtaining images of N ...
Obtaining images of G ...
Obtaining images of Z ...
Obtaining images of T ...
Obtaining images of S ...
Obtaining images of A ...
Obtaining images of F ...
Obtaining images of O ...
Obtaining images of H ...
Obtaining images of del ...
Obtaining images of nothing ...
Obtaining images of space ...
Obtaining images of M ...
Obtaining images of J ...
Obtaining images of C ...
Obtaining images of D ...
Obtaining images of V ...
Obtaining images of Q ...
Obtaining images of X ...
Obtaining images of E ...
Obtaining images of B ...
Obtaining images of K ...
Obtaining images of L ...
Obtaining images of Y ...
Obtaining images of P ...
Obtaining images of W ...


In [7]:
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
           'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [8]:
# Split into Training and Test
import sklearn
from sklearn.model_selection import train_test_split

X = np.array(X)
y = np.array(y)

# Normalize the images (scale pixel values to range [0, 1])
X_normalized = X.astype('float32') / 255.0

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Verify the shapes
print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}, Test labels shape: {y_test.shape}")


Training data shape: (69600, 32, 32, 3), Test data shape: (17400, 32, 32, 3)
Training labels shape: (69600,), Test labels shape: (17400,)


In [9]:
from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
y_train_encoded = to_categorical(y_train, num_classes=len(labels))
y_test_encoded = to_categorical(y_test, num_classes=len(labels))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [19]:
classes = 29
batch = 32
epochs = 1
learning_rate = 0.01

model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dense(classes, activation='softmax'))

adam = Adam(learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train_encoded, batch_size=batch, epochs=epochs, validation_split=0.2, shuffle = True, verbose=1)

1740/1740 [==============================] - 150s 86ms/step - loss: 2.9432 - accuracy: 0.1600 - val_loss: 2.8730 - val_accuracy: 0.1438


In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print('Test Accuracy:', test_acc)
print('Test loss', test_loss)

544/544 [==============================] - 15s 28ms/step - loss: 2.8438 - accuracy: 0.1540
Test Accuracy: 0.1539655178785324
Test loss 2.8437795639038086


In [21]:
model.save('test1.h5')

In [ ]:
import tensorflow as tf
from keras.models import load_model
import numpy as np

# Load the model
model = load_model('test1.h5')


In [23]:
from keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model('atest.h5')

# Define the labels
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
          'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
          'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

# Load the image and preprocess it
img_path = 'ASL_Data/asl_alphabet_train/asl_alphabet_train/A/A329.jpg' 
img = cv2.imread(img_path)

# Resize the image to the target size (32x32)
img_resized = cv2.resize(img, (32, 32))

# Convert the image from BGR to RGB (OpenCV loads images in BGR by default)
img_resized_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)

# Normalize the image (scale pixel values between 0 and 1)
img_resized_rgb = img_resized_rgb / 255.0

# Expand dimensions to match model's input (add batch size dimension)
img_array = np.expand_dims(img_resized_rgb, axis=0)

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index (highest probability)
predicted_class_index = np.argmax(predictions, axis=1)

# Map the predicted index to the corresponding label
predicted_label = labels[predicted_class_index[0]]

# Print the predicted label
print(f"Predicted class: {predicted_label}")


1/1 [==============================] - 0s 59ms/step
Predicted class: U
